In [ ]:
import torch           # This notebook used to train conll finetuned xlmr model with Sinhala English mix(full english set and Sinhala split) for test with Sinhala remaining split
# GPUtil.getAvailable()
use_cuda = torch.cuda.is_available()
use_cuda

True

In [ ]:
# torch.cuda.get_device_name()
device = torch.device("cuda:0" if use_cuda else "cpu")
device

device(type='cuda', index=0)

In [ ]:
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8700
__Number CUDA Devices: 1
__CUDA Device Name: Tesla T4
__CUDA Device Total Memory [GB]: 15.835398144


In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
! pip install datasets seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-p

In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import torch
import pandas as pd
from datasets import load_dataset, load_metric,Dataset ,dataset_dict
import numpy as np
import transformers
from transformers import AutoTokenizer

In [ ]:
task = "ner"
model_checkpoint ="xlm-roberta-base"

In [ ]:
path="/content/drive/Shareddrives/Ram_coders/NER/Data/Final_English.txt"



label_list=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
f1 = open(path, encoding="utf8")
Lines = f1.readlines()
ner_tags=[]
tokens=[]
sen_tok=[]
sen_tag=[]
tags_count={}
for line in Lines:
    if(line!=("\n") and line!=(" \n")):
        try:
            tok,tag=line.split()
            tags_count[tag] = tags_count.get(tag, 0) + 1
            sen_tok.append(tok)
            tag=label_list.index(tag)
            sen_tag.append(tag)
        except:
            print(line,1)
    else:
        ner_tags.append(sen_tag)
        tokens.append(sen_tok)
        sen_tok=[]
        sen_tag=[]
print(len(tokens),len(ner_tags))
df_eng=pd.DataFrame({'tokens':tokens,'ner_tags':ner_tags})
print(tags_count)

3835 3835
{'B-ORG': 1539, 'I-ORG': 3721, 'O': 82335, 'B-MISC': 6283, 'I-MISC': 8587, 'B-LOC': 1511, 'I-LOC': 552, 'B-PER': 194, 'I-PER': 542}


In [ ]:
path="/content/drive/Shareddrives/Ram_coders/NER/Data/Final_Sinhala.txt"




label_list=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
f1 = open(path, encoding="utf8")
Lines = f1.readlines()
ner_tags=[]
tokens=[]
sen_tok=[]
sen_tag=[]
tags_count={}
for line in Lines:
    if(line!=("\n") and line!=(" \n")):
        try:
            tok,tag=line.split()
            tags_count[tag] = tags_count.get(tag, 0) + 1
            sen_tok.append(tok)
            tag=label_list.index(tag)
            sen_tag.append(tag)
        except:
            print(line,1)
    else:
        ner_tags.append(sen_tag)
        tokens.append(sen_tok)
        sen_tok=[]
        sen_tag=[]
print(len(tokens),len(ner_tags))
df_sin=pd.DataFrame({'tokens':tokens,'ner_tags':ner_tags})
print(tags_count)

3835 3835
{'B-ORG': 1628, 'I-ORG': 3766, 'O': 73493, 'B-MISC': 6549, 'I-MISC': 7354, 'B-LOC': 1728, 'I-LOC': 780, 'B-PER': 194, 'I-PER': 610}


In [ ]:
path="/content/drive/Shareddrives/Ram_coders/NER/Data/Final_Tamil.txt"




label_list=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
f1 = open(path, encoding="utf8")
Lines = f1.readlines()
ner_tags=[]
tokens=[]
sen_tok=[]
sen_tag=[]
tags_count={}
for line in Lines:
    if(line!=("\n") and line!=(" \n")):
        try:
            tok,tag=line.split()
            tags_count[tag] = tags_count.get(tag, 0) + 1
            sen_tok.append(tok)
            tag=label_list.index(tag)
            sen_tag.append(tag)
        except:
            print(line,1)
    else:
        ner_tags.append(sen_tag)
        tokens.append(sen_tok)
        sen_tok=[]
        sen_tag=[]
print(len(tokens),len(ner_tags))
df_ta=pd.DataFrame({'tokens':tokens,'ner_tags':ner_tags})
print(tags_count)

3835 3835
{'B-ORG': 1420, 'I-ORG': 2247, 'O': 62947, 'B-MISC': 6012, 'B-LOC': 1829, 'I-LOC': 437, 'I-MISC': 7543, 'B-PER': 226, 'I-PER': 406}


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'data' that you want to split
train_frac = 0.7  # Fraction of data for training
valid_frac = 0.1  # Fraction of data for validation
test_frac = 0.2  # Fraction of data for testing

# Splitting the data into train and remaining data
df_eng_train, remaining_data_en = train_test_split(df_eng, test_size=(valid_frac + test_frac), random_state=42)

# Splitting the remaining data into validation and test sets
df_eng_valid, df_eng_test = train_test_split(remaining_data_en, test_size=(test_frac / (valid_frac + test_frac)), random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'data' that you want to split
train_frac = 0.7  # Fraction of data for training
valid_frac = 0.1  # Fraction of data for validation
test_frac = 0.2  # Fraction of data for testing

# Splitting the data into train and remaining data
df_sin_train, remaining_data_si = train_test_split(df_sin, test_size=(valid_frac + test_frac), random_state=42)

# Splitting the remaining data into validation and test sets
df_sin_valid, df_sin_test = train_test_split(remaining_data_si, test_size=(test_frac / (valid_frac + test_frac)), random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'data' that you want to split
train_frac = 0.7  # Fraction of data for training
valid_frac = 0.1  # Fraction of data for validation
test_frac = 0.2  # Fraction of data for testing

# Splitting the data into train and remaining data
df_ta_train, remaining_data_ta = train_test_split(df_ta, test_size=(valid_frac + test_frac), random_state=42)

# Splitting the remaining data into validation and test sets
df_ta_valid, df_ta_test = train_test_split(remaining_data_ta, test_size=(test_frac / (valid_frac + test_frac)), random_state=42)


In [ ]:
dataset_train=pd.concat([df_sin_train,df_eng_train,df_ta_train]).reset_index(drop=True)
dataset_train

,tokens,ner_tags
0,"[14, (, I, ), 1වන, අරමුණ, :, මුල්, අරමුණෙහි, එ...","[7, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[ඉදිරියේ, දී, මෙම, කොන්ත්‍රාත්, සඳහා, ගිවිසුම්...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[කරුණු, දැක්වීම, විගණන, හා, කළමනාකරණ, කමිටුව, ...","[0, 0, 3, 4, 4, 4, 0, 0, 0]"
3,"[මේ, සමඟ, හින්දු, ආගමික, සාමාන්‍ය, දැනුම, විභා...","[0, 0, 7, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, ..."
4,"[කෘෂි, තාක්ෂණික, ඩිප්ලෝමාව, සහතික, පත්‍ර, පාඨම...","[7, 8, 8, 8, 8, 8, 7, 0, 0, 0, 0]"
...,...,...
8047,"[அந்தத், தரநிலைகள், ,, நிதிநிலை, அறிக்கைகள், ப...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8048,"[2014, ,, ஜுன், 28, ஆம், திகதி, 160, பரீட்சை, ...","[7, 8, 8, 8, 8, 8, 7, 7, 8, 0, 5, 0, 0, 7, 8, ..."
8049,"[இவ்வாறான, குறைபாடுகள், இதன், பின்னர், ஏற்படாத...","[0, 0, 0, 0, 0, 0, 0, 0]"
8050,"[5.10, ., 2014, ஆம், ஆண்டின், 14, ஆம், இலக்கத்...","[7, 0, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, ..."


In [ ]:
dataset_valid=pd.concat([df_sin_valid,df_eng_valid,df_ta_valid]).reset_index(drop=True)
dataset_valid

,tokens,ner_tags
0,"[විශ්‍රාම, ලද, සියලු, දෙනා, මැතිවරණ, දෙපාර්තමේ...","[0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[මෙහි, අදියර, 1, යටතේ, ජල, ශක්ති, යටිතල, පහසුක...","[0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, ..."
2,"[අවසන්, නිවේදනය, නිකුත්, කිරීමෙන්, අනතුරුව, මෙ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[ළමා, හිමිකම්, ප්‍රවර්ධන, නිලධාරීන්ගේ, දිස්ත්‍...","[7, 8, 8, 8, 0, 0, 0, 0, 0]"
4,"[13, ., අවිස්සාවේල්ල, මහාධිකරණයේ, තාප්පය, ඉදි,...","[7, 0, 5, 3, 0, 0, 0]"
...,...,...
1144,"[2015.12.31, ஆம், திகதியில், தொழில், நியாய, சப...","[7, 0, 0, 3, 4, 0, 0, 0, 0, 7, 0, 0, 7, 0, 0, ..."
1145,"[ஏப்ரல், பாணந்துறை, கடற்கரை, விகார, மகாதவேி, ப...","[7, 5, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1146,"[இதன், கீழே, 07, பிரதேச, செயலகங்களின், 08, நிர...","[0, 0, 7, 3, 4, 7, 0, 7, 8, 0, 0, 0]"
1147,"[செயற்றிட்டத்தின், பூர்வாங்க, வேலைகள், முடிக்க...","[0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 0, 0, 7, 0, ..."


In [ ]:
dataset_test=pd.concat([df_sin_test,df_eng_test,df_ta_test]).reset_index(drop=True)
dataset_test

,tokens,ner_tags
0,"[ස්ථානය, 5, වසර, ශිෂ්‍යත්ව, විභාගය, සඳහා, සම්ම...","[0, 7, 0, 0, 0, 0, 0, 0, 3, 4, 0, 5, 0, 5, 0, ..."
1,"[වාරිමාර්ග, පද්ධතිය, මහනුවර, දිස්ත්‍රික්කයේ, ව...","[0, 0, 5, 6, 0, 0, 0, 0, 0, 3, 4, 0, 3, 4, 4, ..."
2,"[2, ., ප්‍රකමනය, කළ, හැකි, තර්කන, පාලක, පාඨමාල...","[7, 0, 7, 8, 8, 8, 8, 8, 0, 7, 0, 7, 0, 7, 8, ..."
3,"[සමාජ, සේවා, අමාත්‍යාංශය, ,, සමාජ, සේවා, ද‍ෙපා...","[3, 4, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[2015.01.01, සිට, 2015.12.31, දක්වා, ප්‍රගතිය,...","[7, 0, 7, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
2299,"[இந்த, மொத்த, ஒதுக்கீடானது, பின்வரும், வகையில்...","[0, 0, 0, 0, 0, 0, 0]"
2300,"[மேலும், ,, 2015, ஆம், ஆண்டில், ஏனைய, அமைச்சுக...","[0, 0, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, ..."
2301,"[இலாபம், பெறுபவரின், எண்ணிக்கை, நடாத்தப்பட்ட, ...","[0, 0, 0, 0, 0, 7, 8, 8, 8, 0, 5, 5, 3, 4, 0, ..."
2302,"[10.2.4, ., 2015.06.21, ஆம், திகதி, தொடக்கம், ...","[7, 8, 7, 0, 0, 0, 7, 0, 0, 0, 5, 6, 0, 7, 8, ..."


In [ ]:
# dataset.iloc[24577].tokens
# dataset.iloc[24577].ner_tags
dataset_train = dataset_train.sample(frac=1).reset_index(drop=True)
dataset_train

,tokens,ner_tags
0,"[Formulation, &, execution, ,, monitoring, &, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[ඒ, මහතා, මැතිවරණ, දෙපාර්තමේන්තුව, වෙනුවෙන්, ඉ...","[0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[(, අ, ), සභාවේ, කර්තව්‍ය, ඉටුකිරීම, සඳහා, අවශ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[(, ඊ, ), ප්‍රාදේශීය, ලේකම්, කාර්යයාල, වලට, යව...","[0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[13, seminars, for, final, exam, for, Aranery,...","[7, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0]"
...,...,...
8047,"[9, ., முதியோர்களை, காக்கும், அர்ப்பணிப்பாளர்,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, ..."
8048,"[17, ., குற்றவியல், நடைமுறை, (, திருத்தச், ), ...","[7, 0, 7, 8, 8, 8, 8, 8, 3, 0, 0, 0, 0, 0, 0]"
8049,"[இவ்வருடத்தில், 3, ஆம், போகமும், உட்பட, 60,000...","[0, 7, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8050,"[ගණන්, කිරීමේ, මධ්‍යස්ථානවලට, අවශ්‍ය, ආරක්ෂාව,...","[0, 0, 0, 0, 0, 3, 0, 0, 0, 0]"


In [ ]:
dataset_valid = dataset_valid.sample(frac=1).reset_index(drop=True)
dataset_valid

,tokens,ner_tags
0,"[This, has, been, also, accurately, stated, in...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[2015, ஆம், ஆண்டிற்காக, மதிப்பிட்ட, வருமான, இல...","[7, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, ..."
2,"[Manual, for, Famous, Temples, and, Guideline,...","[7, 8, 8, 8, 0, 7, 8, 8, 8, 8, 0, 0, 0, 0, 0, ..."
3,"[විශේෂ, අවශ්‍යතා, ඇති, ළමුන්, සඳහා, කල්තියා, ම...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[04, ., CRIMINAL, PROCEDURE, CODE, (, Special,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1144,"[2015.01.01, ஆம், திகதி, தொடக்கம், 2015.12.31,...","[7, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, ..."
1145,"[ආ, ), පාසැල්, භූමිවල, ආරක්ෂාව, සඳහා, නාය, යෑම...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ..."
1146,"[இது, 2012, ஆம், ஆண்டில், மீண்டும், திருத்தப்ப...","[0, 7, 0, 0, 0, 0, 0, 0, 0, 0]"
1147,"[In, 2014, ,, recruitments, ,, making, permane...","[0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
dataset_test = dataset_test.sample(frac=1).reset_index(drop=True)
dataset_test

,tokens,ner_tags
0,"[மேலதிக, செயற்பாடுகள், சில, ஆரம்பிக்கப்பட்டன, .]","[0, 0, 0, 0, 0]"
1,"[12, ., தேசிய, போட்டித்தன்மை, நியமத்தை, தரமுயர...","[0, 0, 0, 0, 0, 0, 0]"
2,"[02, ., ජල, සම්පත්, හා, වාරි, පද්ධති, සංවර්ධනය]","[7, 0, 0, 0, 0, 0, 0, 0]"
3,"[රජයේ, මූල්‍ය, ක්‍රියා, පටිපාටිය, වඩාත්, නිවැර...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, ..."
4,"[2015, ஆம், ஆண்டில், 111, பிள்ளைகள், பதிவு, செ...","[7, 8, 8, 7, 8, 0, 0, 0]"
...,...,...
2299,"[මෙම, කෙටුම්පතට, තවදුරටත්, ඇතුළත්, කළ, යුතු, ක...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2300,"[பிரதான, எலவின், நீளம், 12.8, கி.மீ, ஆவதுடன், ...","[0, 0, 0, 7, 8, 0, 0, 0, 7, 8, 0, 0]"
2301,"[9.5.4, ., பொது, முகாமைத்துவ, உதவியாளர், சேவைய...","[7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ..."
2302,"[A, cost, of, Rs, ., 1,122,350.00, was, incurr...","[0, 0, 0, 7, 8, 8, 0, 0, 0, 7, 0, 0, 0, 0]"


In [ ]:
label_list=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train = Dataset.from_pandas(dataset_train)

In [ ]:
valid = Dataset.from_pandas(dataset_valid)

In [ ]:
test = Dataset.from_pandas(dataset_test)

In [ ]:
tokenized_datasets_train = train.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/8052 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets_valid = valid.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets_test = test.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2304 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list)).to(device=device).to('cuda:0')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-33-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
from seqeval.metrics import classification_report, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# t=tokenized_datasets['train']
# e=tokenized_datasets['test']

In [ ]:
args = TrainingArguments(
    output_dir =f"test-{task}",
    overwrite_output_dir =True,
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    seed=42,
    save_total_limit  =3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.342500,0.329812,0.627935,0.639979,0.633900,0.881532
2,0.240500,0.313826,0.674958,0.701918,0.688174,0.897505
3,0.180000,0.320712,0.675786,0.730248,0.701962,0.897683


TrainOutput(global_step=3021, training_loss=0.28348756199520736, metrics={'train_runtime': 883.5639, 'train_samples_per_second': 27.339, 'train_steps_per_second': 3.419, 'total_flos': 1261792066959072.0, 'train_loss': 0.28348756199520736, 'epoch': 3.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets_test)

{'eval_loss': 0.29016363620758057,
 'eval_precision': 0.6992178424843182,
 'eval_recall': 0.734602554714832,
 'eval_f1': 0.716473575622917,
 'eval_accuracy': 0.9083776404655841,
 'eval_runtime': 23.5585,
 'eval_samples_per_second': 97.799,
 'eval_steps_per_second': 6.112,
 'epoch': 3.0}

In [ ]:
from seqeval.metrics import classification_report

predictions, labels, _ = trainer.predict(tokenized_datasets_test)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

classification_report_result = classification_report(true_labels, true_predictions)

print("Classification report:")
print(classification_report_result)

Classification report:
              precision    recall  f1-score   support

         LOC       0.75      0.86      0.80      2792
        MISC       0.70      0.69      0.70      7276
         ORG       0.60      0.70      0.65      1834
         PER       0.72      0.74      0.73       389

   micro avg       0.70      0.73      0.72     12291
   macro avg       0.69      0.75      0.72     12291
weighted avg       0.70      0.73      0.72     12291



In [ ]:
#for get accuray and IOB tagging wise results
from sklearn.metrics import classification_report

predictions, labels, _ = trainer.predict(tokenized_datasets_test)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels_flat = [label for sent_labels in true_labels for label in sent_labels]
predicted_labels_flat = [label for sent_labels in true_predictions for label in sent_labels]

# Print classification report
print(classification_report(true_labels_flat, predicted_labels_flat))


              precision    recall  f1-score   support

       B-LOC       0.79      0.89      0.84      2791
      B-MISC       0.81      0.77      0.79      7250
       B-ORG       0.66      0.75      0.71      1812
       B-PER       0.77      0.78      0.77       385
       I-LOC       0.61      0.70      0.65       772
      I-MISC       0.76      0.75      0.75      8756
       I-ORG       0.73      0.85      0.78      3284
       I-PER       0.91      0.88      0.90       792
           O       0.96      0.95      0.95     78543

    accuracy                           0.91    104385
   macro avg       0.78      0.81      0.79    104385
weighted avg       0.91      0.91      0.91    104385



In [ ]:
#  trainer.save_model('xlm-roberta-base-finetuned-tamil-english-mixed-for-testwithtamil')
trainer.save_model('xlm-roberta-base-conll-finetuned-finetuned-sinhala-english-mixed')

In [ ]:
# !cp  "/content/xlm-roberta-base-finetuned-tamil-english-mixed-for-testwithtamil" -r  "/content/drive/MyDrive/Colab Notebooks/FYP/XLM-R/Tamil/models"
!cp  "/content/xlm-roberta-base-conll-finetuned-finetuned-sinhala-english-mixed" -r  "/content/drive/MyDrive/NER_and_Noising"